# **Citi Bike Data Engineering  - EDA - Outlier and Range Validation** 

#### Python Package

In [13]:
import pandas as pd
import numpy as np
import csv
import sys
import os
import importlib
from itertools import combinations
from difflib import get_close_matches
import datetime as dt

#### Python Scripts

In [2]:
# Dynamically add the project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from scripts.create_markdown_table import create_markdown_table

#### Import DataFrame

In [3]:
df_dir = os.path.join(project_root, 'eda', 'dataframes')
newark_airport_df = pd.read_pickle(os.path.join(df_dir, 'newark_airport_df.pkl'))
citibike_df = pd.read_pickle(os.path.join(df_dir, 'citibike_df.pkl'))

# Confirm that the dataframes are loaded correctly
if newark_airport_df.empty or citibike_df.empty:
    raise ValueError("One or both dataframes are empty. Please check the data loading process.")
else:
    print("Dataframes loaded successfully.")

Dataframes loaded successfully.


#### Outline Business Logic Constraints

In [14]:
# Create a dictionary that sets logical bounds for newark_airport_df
newark_airport_logical_bounds = {
    'DATE': (dt.date(2016, 1, 1), dt.date(2016, 12, 31)),
    'AWND':(0,100), # Assuming average wind speed in mph
    'SNOW': (0, 10),  # Assuming snowfall is typically not more than 10 inches in a day
    'SNWD': (0, 50),  # Assuming snow depth is typically not more than 50 inches
    'TMAX': (0, 120),  # Assuming maximum temperature should not exceed 120°F
    'TMIN': (-30, 80),  # Assuming minimum temperature should not be below -30°F
    'TAVG': (-30, 120),  # Assuming average temperature should be within a reasonable range
    'PRCP': (0, 5),  # Assuming precipitation is typically not more than 5 inches in a day
    'WDF2': (0, 360),  # Wind direction in degrees
    'WDF5': (0, 360),  # Wind direction in degrees
    'WSPM': (0, 100),  # Assuming wind speed in mph
    'WSF2': (0, 100),  # Assuming wind speed in mph
    'WSF5': (0, 100)  # Assuming wind speed in mph
}

In [24]:
# Crete a dictionary that sets logical bounds for citibike_df
citibike_logical_bounds = {
    'Start Station Latitude': (40.5, 41.0),  # Approximate bounds for NYC
    'Start Station Longitude': (-75.0, -72.0),  # Approximate bounds for NYC
    'End Station Latitude': (40.5, 41.0),  # Approximate bounds for NYC
    'End Station Longitude': (-75.0, -72.0),  # Approximate bounds for NYC
    'Start Time': (pd.Timestamp('2016-01-01 00:00:00'), pd.Timestamp('2016-12-31 23:59:59')),
    'End Time': (pd.Timestamp('2016-01-01 00:00:00'), pd.Timestamp('2016-12-31 23:59:59')),
    'Trip Duration': (0, 86400),  # Assuming trip duration is in seconds and should not exceed 24 hours
    'Birth Year': (1916, 2000), # Assuming participants are within a reasonable age range
}

In [ ]:
# Create a dictionary for business context related to wind speeds
wind_speed_bounds_data = [
    {"Description": "Calm to Light Breeze", "min_wind_speed": 0, "max_wind_speed": 10, "Notes": "Common daily conditions"},
    {"Description": "Moderate to Strong Breeze", "min_wind_speed": 10, "max_wind_speed": 40, "Notes": "Typical during gusty days or storms"},
    {"Description": "Gale to Severe Gale", "min_wind_speed": 40, "max_wind_speed": 74, "Notes": "Rare but possible in strong weather systems"},
    {"Description": "Hurricane-force Winds", "min_wind_speed": 75, "max_wind_speed": 100, "Notes": "Extreme, often coastal or storm-driven"}
]
wind_speed_df = pd.DataFrame(wind_speed_bounds_data)

In [ ]:
# Create a dictionary for business context related to temperature bounds in New Jersey
nj_temperature_bounds_data = [
    {"Description": "Frigid Winter Lows", "min_temp_f": -20, "max_temp_f": 20, "Notes": "Rare but possible in inland or northern NJ during deep winter"},
    {"Description": "Typical Winter Range", "min_temp_f": 21, "max_temp_f": 40, "Notes": "Common from December to February"},
    {"Description": "Mild Spring/Fall", "min_temp_f": 41, "max_temp_f": 65, "Notes": "Transitional seasons, typical March–May and October–November"},
    {"Description": "Warm Summer Days", "min_temp_f": 66, "max_temp_f": 85, "Notes": "Typical June–August daytime highs"},
    {"Description": "Extreme Summer Heat", "min_temp_f": 86, "max_temp_f": 105, "Notes": "Occasional heatwaves, especially in urban areas"}
    ]

nj_temp_df = pd.DataFrame(nj_temperature_bounds_data)

In [ ]:
# Create a function that check the business logical bounds for a given dataframe
def apply_business_rules(df, logical_bounds, date_columns=None):
    """
    Checks if values in specified columns fall outside given logical bounds.
    Optionally converts date columns to datetime and filters invalid values.
    """
    violations = []
    df = df.copy()

    # Convert specified date columns to datetime and drop invalid entries
    if date_columns:
        for date_col in date_columns:
            if date_col in df.columns:
                df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
                df = df[df[date_col].notna()]

    for col, bounds in logical_bounds.items():
        if col not in df.columns:
            continue

        lower, upper = bounds

        # Evaluate out-of-bounds values only where data is present
        mask = df[col].notna() & ((df[col] < lower) | (df[col] > upper))
        if mask.any():
            temp_df = pd.DataFrame({
                "Violated Column": col,
                "Lower Bound": lower,
                "Upper Bound": upper,
                "Identified Value": df.loc[mask, col]
            }).reset_index(drop=True)
            violations.append(temp_df)

    if violations:
        return pd.concat(violations, ignore_index=True)
    else:
        return pd.DataFrame(columns=["Violated Column", "Lower Bound", "Upper Bound", "Identified Value"])

# Apply business rules to both dataframes
newark_airport_violations = apply_business_rules(newark_airport_df, newark_airport_logical_bounds)

citibike_violations = apply_business_rules(citibike_df, citibike_logical_bounds, ['Start Time', 'Stop Time'])

# Create a markdown table for the violations in newark_airport_df
if not newark_airport_violations.empty:
    newark_airport_violations_md = create_markdown_table(newark_airport_violations, "newark_airport_logic_violations")

# Create a markdown table for the violations in citibike_df
if not citibike_violations.empty:
    citibike_violations_md = create_markdown_table(citibike_violations, "citibike_logic_violations")




In [34]:
# Create a function to validate trip duration against start and end timestamps
def validate_trip_duration_against_timestamps(df, duration_column, start_col, end_col, tolerance=1, min_duration=0, max_duration=86400):

    # Ensure datetime types
    df = df.copy()
    # Check if the spcified columns is of the datetime64 type, if not convert it
    if not pd.api.types.is_datetime64_any_dtype(df[start_col]):
        df[start_col] = pd.to_datetime(df[start_col])
    if not pd.api.types.is_datetime64_any_dtype(df[end_col]):
        df[end_col] = pd.to_datetime(df[end_col])

    # Calculate actual duration in seconds
    df["Computed Duration"] = (df[end_col] - df[start_col]).dt.total_seconds()

    # Flag any mismatches with the added tolerance
    mask = (
        (df[duration_column] != df["Computed Duration"]).abs() > tolerance |
        (df["Computed Duration"] < min_duration) |
        (df["Computed Duration"] > max_duration)
    )

    violations = df.loc[mask, [start_col, end_col, duration_column, "Computed Duration"]].copy()
    violations["Violated Column"] = "Trip Duration (derived)"
    violations["Expected Range"] = f"{min_duration}-{max_duration} seconds"

    return violations.reset_index(drop=True)

citibike_duration_violations = validate_trip_duration_against_timestamps(citibike_df,'Trip Duration','Start Time','Stop Time')

# Create a markdown table for the trip duration violations
if not citibike_duration_violations.empty:
    citibike_duration_violations_md = create_markdown_table(citibike_duration_violations, "citibike_trip_duration_violations")


#### Detect Categorical and Quantitative Outliers

In [ ]:
# Create a function to calculate quantitative outlier distributions, using the IQR method and Z-scores
def quantitative_outlier_distribution(df, iqr_factor=1.5, z_thresh=3):
    """
    For each numeric column in the dataframe:
    - Calculate IQR bounds and outlier count
    - Count of Q1 and Q2 quartile values
    - Mean and Std
    - Z-score thresholds and counts
    - Returns a summary dataframe with each metric in a separate column
    """
    summary = []
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        s = df[col].dropna()
        if s.empty:
            continue

        # IQR stats
        Q1 = s.quantile(0.25)
        Q2 = s.quantile(0.50)
        Q3 = s.quantile(0.75)
        IQR = Q3 - Q1
        iqr_lower = Q1 - iqr_factor * IQR
        iqr_upper = Q3 + iqr_factor * IQR

        # IQR segment counts
        q1_to_q2 = s[(s >= Q1) & (s < Q2)].count()
        q2_to_q3 = s[(s >= Q2) & (s <= Q3)].count()
        iqr_outliers = s[(s < iqr_lower) | (s > iqr_upper)].count()

        # Z-score bounds
        mean = s.mean()
        std = s.std()
        z_lower = mean - z_thresh * std
        z_upper = mean + z_thresh * std

        z_inlier_count = s[(s >= z_lower) & (s <= z_upper)].count()
        z_outlier_count = s[(s < z_lower) | (s > z_upper)].count()

        summary.append({
            "Column": col,
            "IQR Range": f"[{round(Q1, 2)}-{round(Q3, 2)}]",
            "Q1-Q2": q1_to_q2,
            "Q2-Q3": q2_to_q3,
            "IQR Outliers": iqr_outliers,
            "Mean": round(mean, 2),
            "Std": round(std, 2),
            "Z-Score Range": f"[{round(z_lower, 2)}-{round(z_upper, 2)}]",
            "Z-Score Inliers": z_inlier_count,
            "Z-Score Outliers": z_outlier_count
        })

    return pd.DataFrame(summary)

# Apply the quantitative outlier bins function to both dataframes
newark_airport_quantitative_outliers = quantitative_outlier_distribution(newark_airport_df)
citibike_quantitative_outliers = quantitative_outlier_distribution(citibike_df)

# Create markdown tables for the quantitative outlier distributions
if not newark_airport_quantitative_outliers.empty:
    newark_airport_quantitative_outliers_md = create_markdown_table(newark_airport_quantitative_outliers, "newark_airport_quantitative_outliers")
if not citibike_quantitative_outliers.empty:
    citibike_quantitative_outliers_md = create_markdown_table(citibike_quantitative_outliers, "citibike_quantitative_outliers")


In [41]:
# Create a function to calculate categorical outlier distributions, using frequency counts
def categorical_outlier_distribution(df, top_n=3, rare_threshold=0.01):
    """
    Summarizes each categorical column with:
    - Unique count
    - Top frequent categories
    - Rare category count
    - Dominance of top class
    """
    summary = []

    categorical_cols = df.select_dtypes(include=["object", "category"]).columns

    for col in categorical_cols:
        s = df[col].dropna()
        total = len(s)
        if total == 0:
            continue

        value_counts = s.value_counts(normalize=True)
        absolute_counts = s.value_counts()

        # Top categories
        top_categories = ", ".join([f"{idx} ({round(pct*100, 1)}%)"
                                    for idx, pct in value_counts.head(top_n).items()])

        # Rarity
        rare_count = (value_counts < rare_threshold).sum()

        summary.append({
            "Column": col,
            "Unique Categories": s.nunique(),
            f"Top {top_n} Categories": top_categories,
            "Dominant Category (%)": round(value_counts.iloc[0]*100, 1),
            f"Rare Categories (< {int(rare_threshold*100)}%)": rare_count
        })

    return pd.DataFrame(summary)

# Apply the categorical outlier bins function to both dataframes
newark_airport_categorical_outliers = categorical_outlier_distribution(newark_airport_df)
citibike_categorical_outliers = categorical_outlier_distribution(citibike_df)

citibike_categorical_outliers

,Column,Unique Categories,Top 3 Categories,Dominant Category (%),Rare Categories (< 1%)
0,Start Time,244407,"2016-09-28 08:24:23 (0.0%), 2016-08-16 08:35:3...",0.0,244407
1,Stop Time,244137,"2016-04-17 17:33:34 (0.0%), 2016-10-01 15:30:2...",0.0,244137
2,Start Station Name,51,"Grove St PATH (11.6%), Exchange Place (7.7%), ...",11.6,22
3,End Station Name,102,"Grove St PATH (15.5%), Exchange Place (9.0%), ...",15.5,76
4,User Type,2,"Subscriber (93.7%), Customer (6.3%)",93.7,0
5,Trip Date,362,"2016-09-13 (0.6%), 2016-09-22 (0.5%), 2016-09-...",0.6,362
